In [ ]:
#!pip install -U xarray

In [19]:
import earthaccess
import rasterio
import xarray as xr

In [2]:
assert(xr.__version__ >= '2024.10.0') # for datatree support

In [3]:
earthaccess.login()

Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ········


In [16]:
s3fs = earthaccess.get_s3_filesystem(daac="LPDAAC")

In [21]:
url = 's3://lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T58FEK_20230223T224809Z_20231222T123638Z_S2B_30_v1/OPERA_L3_DIST-ALERT-HLS_T58FEK_20230223T224809Z_20231222T123638Z_S2B_30_v1_VEG-DIST-STATUS.tif'
#ds = xr.open_dataset(s3fs.open(url))
ds = rasterio.open(s3fs.open(url, "rb"))

In [31]:
ds.bounds
tms = morecantile.tms.get("WebMercatorQuad")

In [35]:
for tileset in tms.tiles(*ds.bounds, zooms=[1]):
    print(tileset)

Tile(x=0, y=0, z=1)
Tile(x=1, y=0, z=1)
Tile(x=0, y=1, z=1)
Tile(x=1, y=1, z=1)


In [39]:
import rasterio
from rasterio.warp import transform_bounds
import morecantile
import math

with rasterio.open(s3fs.open(url, "rb")) as src:
    tms = morecantile.tms.get("WebMercatorQuad")
    
    # Get native resolution
    pixel_size = src.res[0]
    
    # Transform bounds to WGS84
    bounds_4326 = transform_bounds(
        src.crs, 'EPSG:4326',
        src.bounds.left, src.bounds.bottom,
        src.bounds.right, src.bounds.top
    )
    
    center_lon = (bounds_4326[0] + bounds_4326[2]) / 2
    center_lat = (bounds_4326[1] + bounds_4326[3]) / 2
    
    # Find best zoom
    best_zoom = None
    best_diff = float('inf')
    
    for zoom in range(8, 19):
        tile_res = (156543.03 * math.cos(math.radians(center_lat))) / (2 ** zoom)
        diff = abs(tile_res - pixel_size)
        if diff < best_diff:
            best_diff = diff
            best_zoom = zoom
    
    # Get center tile
    tile = tms.tile(center_lon, center_lat, best_zoom)
    
    print(f"Resolution: {pixel_size:.2f} m/pixel")
    print(f"Best zoom: {best_zoom}")
    print(f"Sample tile: z={tile.z}, x={tile.x}, y={tile.y}")
    print(f"URL: /{tile.z}/{tile.x}/{tile.y}")

Resolution: 30.00 m/pixel
Best zoom: 12
Sample tile: z=12, x=3934, y=2709
URL: /12/3934/2709
